In [33]:
import pandas as pd
import numpy as np


import pickle

from itertools import chain

from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV, LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

import pickle

In [34]:
train = pd.read_csv('../../data/engineered/training.csv')
submission_input = pd.read_csv('../../data/engineered/test.csv')

target = 'SalePrice'
y = train[target]
X = train[[col for col in train.columns if col != target]]
y_log = np.log1p(y)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=.3, random_state=41)

In [36]:
n_iter = 100000
alphas = np.random.exponential(1, 10000)

lasso_model = LassoCV(alphas=alphas, max_iter=n_iter, cv=4)
lasso_model.fit(X_train, y_train)

LassoCV(alphas=array([ 0.76237,  1.08792, ...,  0.15739,  2.69249]),
    copy_X=True, cv=4, eps=0.001, fit_intercept=True, max_iter=100000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [37]:
lasso_model.score(X_test, y_test)

0.84848408476909687

In [38]:
mean_squared_error(y_test, lasso_model.predict(X_test))**.5

0.15375385265093805

In [39]:
mean_squared_error(np.exp(y_test) - 1, np.exp(lasso_model.predict(X_test)) - 1)**.5

31694.639746185156

In [72]:
with open('lasso.pickle', "wb") as pickle_model:
    pickle.dump(lasso_model, pickle_model)


# with open('lasso.pickle', "rb") as pickle_model:
#     load_model = pickle.load(pickle_model)

In [40]:
lasso_pred = lasso_model.predict(submission_input)
lasso_pred = np.exp(lasso_pred) - 1
submission = pd.DataFrame(data=
                         {
                             'SalePrice':lasso_pred
                         },
                         index=submission_input.index)
submission.sort_index().to_csv('lasso_submission.csv')

In [41]:
alphas = np.random.exponential(1, 10000)

ridge_model = RidgeCV(alphas=alphas, cv=4)
ridge_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.00953,  1.99174, ...,  1.60981,  0.03694]), cv=4,
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [42]:
ridge_model.score(X_test, y_test)

0.79849070905736852

In [43]:
mean_squared_error(y_test, ridge_model.predict(X_test))**.5

0.1773145676665018

In [44]:
mean_squared_error(np.exp(y_test) - 1, np.exp(ridge_model.predict(X_test)) - 1)**.5

37079.634623962047

In [77]:
with open('ridge.pickle', "wb") as pickle_model:
    pickle.dump(ridge_model, pickle_model)


# with open('ridge.pickle', "rb") as pickle_model:
#     load_model = pickle.load(pickle_model)

In [46]:
ridge_pred = ridge_model.predict(submission_input)
ridge_pred = pd.DataFrame(data=
                         {
                             'SalePrice':ridge_pred
                         },
                         index=submission_input.index)
submission = np.exp(submission) - 1
submission.sort_index().to_csv('ridge_submission.csv')

In [54]:
l1_ratio = [*chain(np.linspace(.01, .3, 1000), np.linspace(.6,1, 9000))]
max_iter = 1000

elastic_model = ElasticNetCV(l1_ratio=l1_ratio, max_iter=max_iter)
elastic_model.fit(X_train, y_train)
elastic_model.score(X_test, y_test)

0.79695872213775809

In [55]:
elastic_model.score(X_test, y_test)

0.79695872213775809

In [56]:
mean_squared_error(y_test, elastic_model.predict(X_test))**.5

0.17798731395409959

In [57]:
mean_squared_error(np.exp(y_test) - 1, np.exp(elastic_model.predict(X_test)) - 1)**.5

39756.232459615458

In [79]:
with open('ridge.pickle', "wb") as pickle_model:
    pickle.dump(ridge_model, pickle_model)


# with open('ridge.pickle', "rb") as pickle_model:
#     load_model = pickle.load(pickle_model)

In [80]:
elastic_pred = elastic_model.predict(submission_input)
elastic_pred = np.exp(elastic_pred) - 1

submission = pd.DataFrame(data=
                         {
                             'SalePrice':elastic_pred
                         },
                         index=submission_input.index)
submission.sort_index().to_csv('elastic_submission.csv')